In [385]:
import pandas

tr = pandas.read_csv('train.csv')
te = pandas.read_csv('test.csv')
prd = pandas.read_csv('pred.csv')
get_dirty = lambda df:df[(df.uid == None) | (df.iid == None) | (df.score == None) | (df.score > 5.0) | (df.score < 1.0)]
print(get_dirty(tr),get_dirty(te),get_dirty(prd))
#输出发现没有脏数据
cut = lambda df:df[['uid','iid','score']].drop_duplicates().dropna()
#丢弃重复和nan的数值，只保留前三列
tr =cut(tr)
te = cut(te)
#输出uid和iid的最大值，用于后续代码使用
print(tr[['uid']].max())
print(tr[['iid']].max())

Empty DataFrame
Columns: [uid, iid, score, review, date]
Index: [] Empty DataFrame
Columns: [uid, iid, score, review, date]
Index: [] Empty DataFrame
Columns: [uid, iid, score, review, date]
Index: []
uid    1925
dtype: int64
iid    789
dtype: int64


In [411]:
import numpy as np
import random as rd
from collections import defaultdict
from math import sqrt

#基于用户的协同过滤，参数含义在主函数处可见解释
def ucf_pred(m,i2u,u2i,u2m,sim_func,k,ms,uid,iid):
    users = i2u[iid] #评价过物品的所有用户
    if len(users) == 0:
        return ms
    #对于不存在的物品，直接返回所有评分的平均值。实验证明这里若考虑物品不存在用户存在的情况，返回u2m[uid]后RMSE剧增
    mean_all = np.mean([m[suid][iid] for suid in users]) #评价过物品所有用户的均值
    if not uid in u2i:
        return mean_all #用户不存在
    sf = lambda x:sim_func(m,u2i,x,uid)
    if len(users) > k:
        users = sorted(users,key = sf,reverse = 1)[:k] #users大于k时根据相似度重排取相似度最大的几个
    r = u2m[uid] #“普遍”的预测函数
    a,b = 0.0,0.0
    for suid in users:
        a += sf(suid) * (m[suid][iid] - u2m[suid]) 
        b += sf(suid)
    if b == 0:
        return mean_all
    return r + a / b

#直接返回平均值，用作参照
def mean_pred(m,i2u,u2i,u2m,sim_func,k,ms,uid,iid):
    users = i2u[iid]
    if len(users) == 0:
        return ms
    return np.mean([m[suid][iid] for suid in users])

#余弦相似度，同时考虑正例与负例，二者取平均
def cos_sim_prototype(m,u2i,uid1,uid2,cmp):
    assert(uid1 in u2i and uid2 in u2i)
    is1,is2 = u2i[uid1],u2i[uid2]
    pos1,pos2 = set([iid for iid in is1 if cmp(m[uid1][iid],3.0)]),set([iid for iid in is2  if cmp(m[uid2][iid],3.0)])
    overlap = pos1.intersection(pos2)
    b,c = 0.0,0.0
    for iid in pos1:
        b += m[uid1][iid] * m[uid1][iid]
    for iid in pos2:
        c += m[uid2][iid] * m[uid2][iid]
    if b==0 or c==0:
        return 0 #对于分母为0的情况返回0
    return len(overlap) / (sqrt(b) * sqrt(c))

def cos_sim(m,u2i,uid1,uid2):
    return (cos_sim_prototype(m,u2i,uid1,uid2,lambda a,b:a < b) + cos_sim_prototype(m,u2i,uid1,uid2,lambda a,b:a > b)) / 2

#Jaccard相似度，同时考虑正例与负例，二者取平均
def jaccard_sim_prototype(m,u2i,uid1,uid2,cmp):
    assert(uid1 in u2i and uid2 in u2i)
    is1,is2 = u2i[uid1],u2i[uid2]
    pos1,pos2 = set([iid for iid in is1 if cmp(m[uid1][iid],3.0)]),set([iid for iid in is2  if cmp(m[uid2][iid],3.0)])
    a,b = float(len(pos1.intersection(pos2))),len(pos1.union(pos2))
    if b == 0: #对于分母为0的情况返回0
        return 0
    return a / b

def jaccard_sim(m,u2i,uid1,uid2):
    return (jaccard_sim_prototype(m,u2i,uid1,uid2,lambda a,b:a < b) + jaccard_sim_prototype(m,u2i,uid1,uid2,lambda a,b:a > b)) / 2

#皮尔逊相似度
def pearson_sim(m,u2i,uid1,uid2):
    assert(uid1 in u2i and uid2 in u2i)
    is1,is2 = u2i[uid1],u2i[uid2]
    mean1,mean2 = np.mean([m[uid1][iid] for iid in is1]),np.mean([m[uid2][iid] for iid in is2])
    overlap = is1.intersection(is2)
    a,b,c = 0.0,0.0,0.0
    for iid in overlap:
        a += (m[uid1][iid] - mean1) * (m[uid2][iid] - mean2)
        b += (m[uid1][iid] - mean1) * (m[uid1][iid] - mean1)
        c += (m[uid2][iid] - mean2) * (m[uid2][iid] - mean2)
    if b==0 or c==0:
        return -1 #分母为0返回-1
    return a / (sqrt(b) * sqrt(c))

#皮尔逊相似度正则化
def pearson_norm_sim(m,u2i,uid1,uid2):
    return (pearson_sim(m,u2i,uid1,uid2) + 1) / 2

#根据测试集用RMSE进行评估
def evaluate(te,m,i2u,u2i,u2m,sim_func,k,ms,uid,iid,pred_func):
    rmse = 0.0
    n = 0
    for idx,row in te.iterrows():
        uid,iid,score = int(row['uid']),int(row['iid']),row['score']
        n += 1
        p = int(pred_func(m,i2u,u2i,u2m,sim_func,k,ms,uid,iid) + 0.5) #四舍五入
        rmse += (score - p) * (score - p)
    rmse = sqrt(rmse/n)
    return rmse

#初始化 m存储分数矩阵，k为相似用户数，sim_func为相似度函数，i2u为物品-用户倒排表，u2i为用户-物品倒排表,pred_func表示预测函数
m = [[np.nan for j in range(789 + 1)] for i in range (1925 + 1)]
k = 1
sim_func = jaccard_sim
pred_func = ucf_pred
i2u = defaultdict(set)
u2i = defaultdict(set)

#计算所有评分的平均数ms，用于预测完全未知的用户
n = 0
ms = 0.0
for idx,row in tr.iterrows():
    uid,iid,score = int(row['uid']),int(row['iid']),row['score']
    m[uid][iid] = score
    ms += score
    n += 1
    i2u[iid].add(uid)
    u2i[uid].add(iid)
ms /= n

#u2m存储用户对应的得分平均值
u2m = defaultdict(float)
for uid in u2i:
    utm[uid]=np.mean([m[uid][iid] for iid in u2i[uid]])

#网格测试，并输出平均对照
'''
mx = 0
v = 999
for i in range(1,21):
    if evaluate(te,m,i2u,u2i,u2m,sim_func,i,ms,uid,iid,pred_func)<v:
        mx,v=i,evaluate(te,m,i2u,u2i,u2m,sim_func,i,ms,uid,iid,pred_func)
print(mx,v)
print(evaluate(te,m,i2u,u2i,u2m,sim_func,k,ms,uid,iid,mean_pred))
'''

#填写答案
ans = []
for idx,row in prd.iterrows():
    uid,iid = int(row['uid']),int(row['iid'])
    ans.append(int(ucf_pred(m,i2u,u2i,u2m,sim_func,k,ms,uid,iid) + 0.5) + 0.0) #四舍五入
prd['score']=ans
prd.to_csv('my_answer.csv')

In [399]:
# mean_pred结果:1.2779839588573316
# cos_sim的最佳结果:k=12 RMSE=1.126185012742946
# jaccard_sim的最佳结果k=1 RMSE=1.121764644162269
# pearson_norm_sim的最佳结果k=3 RMSE=1.2319152471463943
# 由以上网格测试过程可得最佳算法为k=1的jaccard_sim